In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn import svm
from random import randint, random, shuffle
from statistics import mean
import matplotlib.pyplot as plt

In [2]:
LEN = 137

### funkcje do przetwarzania danych

In [3]:
def load_data():
    people = []
    for i in range(1,16):
    # omijam osobę 5
        if i != 5:
#             person = np.genfromtxt('ICA/Features_os' + str(i) + '.csv', delimiter=',')
            person = np.genfromtxt('MCSB_dane/Features_os' + str(i) + '.csv', delimiter=',')
            # delete first row with electrode/signal names
            person = np.delete(person, 0, 0)
            people.append(person)
#   za każdym razem inne osoby służą do treningu, a inne do testu
    shuffle(people)
    rest = np.concatenate((people[0:12]))
    test = np.concatenate((people[12:]))
    np.random.shuffle(rest)
    np.random.shuffle(test)
    return rest, test

In [4]:
def load_data2():
    people = []
    for i in range(15):
    # omijam osobę 5
        if i != 4:
            person = np.genfromtxt('nothing/Features_os' + str(i) + '.csv', delimiter=',')
            # delete first row with electrode/signal names
            person = np.delete(person, 0, 0)
            people.append(person)
#   za każdym razem inne osoby służą do treningu, a inne do testu
    shuffle(people)
    rest = np.concatenate((people[0:12]))
    test = np.concatenate((people[12:]))
    np.random.shuffle(rest)
    np.random.shuffle(test)
    return rest, test

# r, t = load_data2()
# idx = [True if x<34 or x==136 else False for x in range(137)]
# r = r[:,idx]
# t = t[:,idx]

In [5]:
# funkcje pomocnicze zmieniające wartość w ostatniej kolumnie arraya

def change_4_to_0(arr):
    if arr[-1] == 4:
        arr[-1] = 0
        
def change_5_to_1(arr):
    if arr[-1] == 5:
        arr[-1] = 1
        
def change_5_to_0(arr):
    if arr[-1] == 5:
        arr[-1] = 0

In [6]:
# funkcja zmieniająca wartość w ostatniej kolumnie w zależności od tego czy klasyfikator ma odróżniać:
# - błędy interfejsu od udanych kliknięć i błędów użytkownika (do bci uczącego się)
# - błędy interfejsu i błędy użytkownika od udanych kliknięć (do bci cofającego błędy)

def recode_data(data, bci_type):
    if bci_type == 'learning':
#         błędy użytkownika (5) traktowane jak sukcesy
        np.apply_along_axis(change_4_to_0, 1, data)
        np.apply_along_axis(change_5_to_1, 1, data)
    elif bci_type == 'withdraw':
#         błędy użytkownika (5) traktowane jak porażki
        np.apply_along_axis(change_4_to_0, 1, data)
        np.apply_along_axis(change_5_to_0, 1, data)
    else:
        print('wrong type')
    
# recode_data(rest, 'withdraw')

In [7]:
# to ostatecznie się nie udało

def coefficient_of_variation(arr):
    return np.std(arr)/np.mean(arr)

def remove_low_variation(rest, test, treshold):
#     print(rest)
    n_rest_rows = rest.shape[0]
    n_test_rows = test.shape[0]
    rest_labels = rest[:,-1]
    test_labels = test[:,-1]
    coef_of_var = np.apply_along_axis(coefficient_of_variation, 0, rest[:,:-1])
#     print(coef_of_var)
    sorted_arr = np.sort(coef_of_var)
#     print(sorted_arr)
#     print(sorted_arr[-treshold:])
#     print(sorted_arr[-treshold])
    smallest_val = sorted_arr[-treshold]
#     print(np.where(coef_of_var >= smallest_val))
    idx = np.where(coef_of_var >= smallest_val)
#     print(rest[:,idx])
    rest = np.hstack((rest[:,idx].reshape(n_rest_rows,treshold), np.expand_dims(rest_labels, axis=1)))
    test = np.hstack((test[:,idx].reshape(n_test_rows,treshold), np.expand_dims(test_labels, axis=1)))
    return rest, test

# np.expand_dims(rest_labels, axis=1)

# b = np.array([[1,2,3,4], [4,5,6,7], [7,8,9,2], [1,2,3,3], [4,5,6,1], [7,8,9,9]])
# c = np.array([[3,4,7,0], [2,1,2,2]])

# result = remove_low_variation(b, c, 2)
# result

In [9]:
# funkcja standaryzująca każdą z cech tak, aby miała średnią = 0 i wariancję = 1

def scale_data(rest, test):
    scaler = preprocessing.StandardScaler().fit(rest[:,:-1])
    scaled_rest = scaler.transform(rest[:,:-1])
    scaled_test = scaler.transform(test[:,:-1])
    rest = np.hstack((scaled_rest, np.expand_dims(rest[:,-1], axis=1)))
    test = np.hstack((scaled_test, np.expand_dims(test[:,-1], axis=1)))
    return rest, test
    
# rest, test = scale_data(rest, test)

In [10]:
# wyrównywanie liczby wierszy dla każdej z klas
# zwraca dane uporządkowane klasami, ale to nie problem, bo potem i tak są one mieszane przed treningiem i testem

def downsample(data):
    # Indicies of each class' observations
    i_class0 = np.where(data[:,-1] == 0)[0]
    i_class1 = np.where(data[:,-1] == 1)[0]

    # Number of observations in each class
    n_class0 = len(i_class0)
    n_class1 = len(i_class1)

    # For every observation of class 0, randomly sample from class 1 without replacement
    i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

    # Join together class 0's target vector with the downsampled class 1's target vector
    joined = np.vstack((data[i_class0], data[i_class1_downsampled]))
    return joined

### funkcje do algorytmu genetycznego

In [12]:
# funkcja tworząca pierwsze pokolenie
# jedynki oznaczają cechy sygnału przekazywane do klasyfikatora

def first_gen(generation_size):
    first_gen = np.random.randint(2, size=(generation_size,LEN-1))
    return first_gen

In [13]:
# funkcja oceniająca przystosowanie danego osobnika

def eval_ind_knn(individual, X_train, X_test, y_train, y_test):
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train[:,individual==1], y_train)
    accuracy = neigh.score(X_test[:,individual==1], y_test)
#     print("accuracy: ",accuracy)
#     print("zeros: ", np.count_nonzero(individual==0))
    return (accuracy*np.count_nonzero(individual==0))+1

In [14]:
def eval_ind_svm(individual, X_train, X_test, y_train, y_test):
    clf = svm.SVC()
    clf.fit(X_train[:,individual==1], y_train)
    accuracy = clf.score(X_test[:,individual==1], y_test)
#     print("accuracy: ",accuracy)
#     print("zeros: ", np.count_nonzero(individual==0))
    return (accuracy*np.count_nonzero(individual==0))+1

In [15]:
# funkcja oceniająca całe pokolenie

def evaluation(X_train, X_test, y_train, y_test, generation, classifier):
#     X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test
    if classifier == 'knn':
        gen_fitness = np.apply_along_axis(eval_ind_knn, 1, generation, X_train, X_test, y_train, y_test)
    elif classifier == 'svm':
        gen_fitness = np.apply_along_axis(eval_ind_svm, 1, generation, X_train, X_test, y_train, y_test)
    else:
        print('wrong classifier')
#     print(gen_fitness)
    best_ind = generation[np.argmax(gen_fitness)]
    best_fit = max(gen_fitness)
    mean_fit = np.mean(gen_fitness)
#     print("fitness", best_fit)
#     print("mean_fitness", mean_fit)
    best_acc = (best_fit-1)/np.count_nonzero(best_ind==0)
#     mean_acc = ()
#     print('acc',best_acc)
#     print("accuracy: ",best_fit/np.count_nonzero(best_ind==0))
#     print("ones: ", np.count_nonzero(best_ind))
#          gen_fitness,   local_n_of_zeros,      global_best_fitness,     global_best_accuracy,       best_individual
    return gen_fitness, np.count_nonzero(best_ind==0), best_fit, (best_fit-1)/np.count_nonzero(best_ind==0), best_ind
#     return gen_fitness, np.count_nonzero(best_ind==0), best_fit, (best_fit-1)/np.count_nonzero(best_ind==0), best_ind, mean_fit

In [16]:
# ruletka

def roulette(generation, gen_fitness, generation_size):
    gen_fitness /= sum(gen_fitness)
    survivors = np.random.choice(generation_size, generation_size, p=gen_fitness)
    return generation[survivors,:]

In [17]:
# funkcja krzyżująca osobniki

def crossover(survivors, generation_size, crossover_prob):
# czy można się pozbyć tego fora?
    for i in range(0,generation_size,2):
        if np.random.rand() <= crossover_prob:
            cut = np.random.randint(1,LEN-2)
            temp = survivors[1,:cut].copy()
            survivors[1,:cut] = survivors[0,:cut]
            survivors[0,:cut] = temp
    return survivors

In [18]:
# funkcja mutująca wybrane geny wybranych osobników z pokolenia

def mutation(individual, mutation_prob):
    if np.random.rand() <= mutation_prob:
        locus = np.random.randint(0,LEN-2)
        individual[locus] = 1-individual[locus]
#     return ind

### algorytm genetyczny

In [19]:
def gen_alg(X_train, X_test, y_train, y_test, generation_size, num_of_generations, mutation_prob,
            crossover_prob, classifier, rand_state):
#     best_fit_list = []
#     mean_fit_list = []
#     best_acc_list = []
    
# inicjalizacja populacji (first generation)
#     print('gen ', 0)   
    generation = first_gen(generation_size)
    
# ocena przystosowania populacji
    global_best_fitness = 0
#     gen_fitness, local_n_of_zeros, global_best_fitness, global_best_accuracy, best_individual, mean_fit = evaluation(X_train, X_test,y_train, y_test, generation, classifier)
    gen_fitness, local_n_of_zeros, global_best_fitness, global_best_accuracy, best_individual = evaluation(X_train, X_test,y_train, y_test, generation, classifier)
#     
#     best_fit_list.append(global_best_fitness)
#     mean_fit_list.append(mean_fit)
#     best_acc_list.append(global_best_accuracy)
#     
#     print('local ',local_best_fitness)
#     print('num of zeros ', local_n_of_zeros)
#     global_best_fitness = local_best_fitness
#     global_best_accuracy = local_best_accuracy
#     print(global_best_fitness)
        
# główna pętla programu
    for i in range(num_of_generations):
#         print('gen ', i+1)   
# ruletka
        survivors = roulette(generation, gen_fitness, generation_size)
# krzyżowanie
        descendants = crossover(survivors, generation_size, crossover_prob)  
# mutacje
        np.apply_along_axis(mutation, 1, descendants, mutation_prob)
# ocena
#         gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, local_best_individual, mean_fit = evaluation(X_train, X_test, y_train, y_test, descendants, classifier)
        gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, local_best_individual = evaluation(X_train, X_test,y_train, y_test, generation, classifier)

#         print('local ',local_best_fitness)
#         print('num of zeros ', local_n_of_zeros)
        if local_best_fitness > global_best_fitness:
            global_best_fitness = local_best_fitness
            global_best_accuracy = local_best_accuracy
            best_individual = local_best_individual
#         print(global_best_fitness)
        #         
#         best_fit_list.append(global_best_fitness)
#         mean_fit_list.append(mean_fit)
#         best_acc_list.append(local_best_accuracy)


        generation = descendants
#     return global_best_fitness, global_best_accuracy, best_individual, best_fit_list, mean_fit_list, best_acc_list
    return global_best_fitness, global_best_accuracy, best_individual

### funkcje do tworzenia wykresów

In [20]:
# generation_size, num_of_generations, crossover_prob, mutation_prob, classifier,
#          bci_type, coef_of_var, treshold, scaling, downsampling, rand_state
# bci_type='learning'
# rest, end_test = load_data()
# recode_data(rest, bci_type)
# recode_data(end_test, bci_type)
# rest, end_test = scale_data(rest, end_test)
# rest = downsample(rest)
# end_test = downsample(end_test)
# # kfold
# kf = KFold(n_splits=20, shuffle=True, random_state=4)
# j = 0
# best_fit_data = []
# mean_fit_data = []
# best_acc_data = []
# for train_index, test_index in kf.split(rest[:,-1]):
#         X_train, X_test = rest[:,:-1][train_index], rest[:,:-1][test_index]
#         y_train, y_test = rest[:,-1][train_index], rest[:,-1][test_index]
# #         global_best_fitness, global_best_accuracy, best_individual, best_fit_list, mean_fit_list, best_acc_list= gen_alg(X_train,
# #                                                                              X_test, y_train,
# #                                                                              y_test,generation_size=30,
# #                                                                              num_of_generations=400,
# #                                                                              mutation_prob=0.01,
# #                                                                              crossover_prob=1, classifier='knn',
# #                                                                              rand_state=4)
# #         print('genetic algorithm number:', j)
#         best_fit_data.append(best_fit_list)
#         mean_fit_data.append(mean_fit_list)
#         best_acc_data.append(best_acc_list)
#         j+=1

In [21]:
# best_fit_data = np.array(best_fit_data)
# mean_fit_data = np.array(mean_fit_data)
# best_acc_data = np.array(best_acc_data)
# print(best_fit_data.shape, mean_fit_data.shape, best_acc_data.shape)

In [22]:
# print(best_fit_data)

In [23]:
# best_fitnesses = np.apply_along_axis(mean, 0, best_fit_data)
# mean_fitnesses = np.apply_along_axis(mean, 0, mean_fit_data)
# best_accuracies = np.apply_along_axis(mean, 0, best_acc_data)

In [24]:
# best_fitnesses.shape[0]

In [25]:
# # a = np.array(best_fit_list)
# # b = np.array(mean_fit_list)
# # c = np.array(best_acc_list)
# # # print(a)
# # # print(np.array(range(c.shape[0])))
# plt.figure(figsize=(15,5))
# plt.xlabel('numer pokolenia')
# plt.ylabel('dopasowanie')
# plt.plot(np.array(range(best_fitnesses.shape[0])),best_fitnesses, label="najlepsze dopasowanie")
# plt.plot(np.array(range(mean_fitnesses.shape[0])),mean_fitnesses, label="średnie dopasowanie w pokoleniu")
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# plt.plot([1, 2, 3], label="test1")
# plt.plot([3, 2, 1], label="test2")
# Place a legend to the right of this smaller subplot.
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# alltime = np.linspace(0, 500, data[0][:,0].size)
# plt.figure(figsize=(15,5))
# plt.xlabel('time [s]')
# plt.ylabel('amplitude [uV]')
# plt.plot(alltime, data[0][:,0])

In [26]:
# plt.figure(figsize=(15,5))
# plt.xlabel('numer pokolenia')
# plt.ylabel('skuteczność')
# plt.plot(np.array(range(best_accuracies.shape[0])), best_accuracies, label="najlepsza skuteczność w pokoleniu")
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [27]:
# plt.figure()

# models = [np.array(best_fit_list), np.array(mean_fit_list), np.array(best_acc_list)]
# names = ['Best fitness',
#          'Mean fitness',
#          'Best accuracy']
# colors = ['red', 'steelblue', 'green']

# for ii, (model, name) in enumerate(zip(models, names), 1):
#     plt.subplot(3, 1, ii)
#     plt.title(name)
#     for sig, color in zip(model, colors):
#         plt.plot(sig, color=color)

# plt.tight_layout()
# plt.show()

### funkcja do treningu

In [28]:
# z pozostałych 80% do treningu:
# podział na 5 grup

def train(X_rest, y_rest, generation_size, num_of_generations, crossover_prob, mutation_prob, classifier, rand_state):
    kf = KFold(n_splits=5, shuffle=True, random_state=rand_state)
    alltime_best_fitness = 0
#     alltime_best_accuracy = 0
#     best_individual = np.array([])

    for train_index, test_index in kf.split(X_rest):
        X_train, X_test = X_rest[train_index], X_rest[test_index]
        y_train, y_test = y_rest[train_index], y_rest[test_index]
        global_best_fitness, global_best_accuracy, best_individual = gen_alg(X_train, X_test, y_train,
                                                                             y_test,generation_size,
                                                                             num_of_generations,
                                                                             mutation_prob,
                                                                             crossover_prob, classifier,
                                                                             rand_state)
#         print('fitness: ',global_best_fitness,"\naccuracy: ", global_best_accuracy, "\nindividual: ", best_individual)
        if global_best_fitness > alltime_best_fitness:
#             alltime_best_fitness = global_best_fitness
#             alltime_best_accuracy = global_best_accuracy
            alltime_best_individual = best_individual
    return alltime_best_individual

### funkcja przeprowadzająca test końcowy

In [29]:
def perform_end_test(X_end_test, y_end_test, alltime_best_individual, classifier, rand_state):
    folds = KFold(n_splits=X_end_test.shape[0], shuffle=True)
    if classifier == 'knn':
        neigh3 = KNeighborsClassifier(n_neighbors=3)
        score = cross_val_score(neigh3, X_end_test[:,alltime_best_individual==1], y_end_test, cv=folds)
    elif classifier == 'svm':
        clf2 = svm.SVC()
        score = cross_val_score(clf2, X_end_test[:,alltime_best_individual==1], y_end_test, cv=folds)
    else:
        print('wrong classifier')
#     print('end test score: ',score)
    print('\nend test score mean: ',score.mean(), '\n')
    return score.mean()

### funkcja main i jej wykorzystanie

In [30]:
def main(generation_size, num_of_generations, crossover_prob, mutation_prob, classifier,
         bci_type, coef_of_var, treshold, scaling, downsampling, rand_state):
    rest, end_test = load_data()
    recode_data(rest, bci_type)
    recode_data(end_test, bci_type)
    if coef_of_var:
        rest, end_test = remove_low_variation(rest, test, treshold)
    if scaling:
        rest, end_test = scale_data(rest, end_test)
    if downsampling:
        rest = downsample(rest)
        end_test = downsample(end_test)
    
    alltime_best_individual = train(rest[:,:-1], rest[:,-1], generation_size, num_of_generations, 
                                    crossover_prob, mutation_prob, classifier, rand_state)
    print(alltime_best_individual)
    end_test_score_mean = perform_end_test(end_test[:,:-1], end_test[:,-1], alltime_best_individual, classifier, rand_state)
    return end_test_score_mean

In [31]:
def main2(generation_size, num_of_generations, crossover_prob, mutation_prob, classifier,
         bci_type, coef_of_var, treshold, scaling, downsampling, rand_state):
#     rest, end_test = load_data2()
    
    r, t = load_data2()
    idx = [True if x<34 or x==136 else False for x in range(137)]
    rest = r[:,idx]
    end_test = t[:,idx]
    
    recode_data(rest, bci_type)
    recode_data(end_test, bci_type)
    if coef_of_var:
        rest, end_test = remove_low_variation(rest, test, treshold)
    if scaling:
        rest, end_test = scale_data(rest, end_test)
    if downsampling:
        rest = downsample(rest)
        end_test = downsample(end_test)
    
    alltime_best_individual = train(rest[:,:-1], rest[:,-1], generation_size, num_of_generations, 
                                    crossover_prob, mutation_prob, classifier, rand_state)
    print(alltime_best_individual)
    end_test_score_mean = perform_end_test(end_test[:,:-1], end_test[:,-1], alltime_best_individual, classifier, rand_state)
    return end_test_score_mean

In [32]:
def main_no_gen_alg(classifier, bci_type, scaling, downsampling, rand_state):
    rest, end_test = load_data2()
    recode_data(rest, bci_type)
    recode_data(end_test, bci_type)
    if scaling:
        rest, end_test = scale_data(rest, end_test)
    if downsampling:
        rest = downsample(rest)
        end_test = downsample(end_test)
    individual = np.ones(136)
#     print(individual)
    end_test_score_mean = perform_end_test(end_test[:,:-1], end_test[:,-1], individual, classifier, rand_state)
    return end_test_score_mean

In [33]:
# for bci_type in ['learning','withdraw']:
#     for classifier in ['svm','knn']:
#         for scaling in [True, False]:
#             result = []
#             print('bci_type:', bci_type, ', classifier:', classifier, ', scaling:', scaling)
#             for i in range(10):
#                 score = main_no_gen_alg(classifier, bci_type, scaling, downsampling=True, rand_state=4)
#                 result.append(score)
#             print(mean(result), '\n')
# print(score)

In [35]:
# ICA
bci_type = 'learning'
classifier = 'svm'
scaling = True
coef_of_var = False
mutation_prob = 0.01
num_of_generations = 100
generation_size = 30

print('bci_type:', bci_type, ', classifier:', classifier, ', scaling:', scaling, ', coef_of_var:', coef_of_var)
print('mutation_prob:', mutation_prob, ', num_of_generations:', num_of_generations, ', generation_size:', generation_size, '\n')
result = []
for i in range(50):
    score = main(generation_size=generation_size, num_of_generations=num_of_generations, crossover_prob=1, mutation_prob=mutation_prob, 
                classifier=classifier, bci_type=bci_type, coef_of_var = False, treshold = 60,
                scaling=scaling, downsampling=True, rand_state=4)
    result.append(score)
print(mean(result), '\n')

bci_type = 'withdraw'
classifier = 'svm'
scaling = True
coef_of_var = False
mutation_prob = 0.01
num_of_generations = 100
generation_size = 30

print('bci_type:', bci_type, ', classifier:', classifier, ', scaling:', scaling, ', coef_of_var:', coef_of_var)
print('mutation_prob:', mutation_prob, ', num_of_generations:', num_of_generations, ', generation_size:', generation_size, '\n')
result = []
for i in range(51):
    score = main(generation_size=generation_size, num_of_generations=num_of_generations, crossover_prob=1, mutation_prob=mutation_prob, 
                classifier=classifier, bci_type=bci_type, coef_of_var = False, treshold = 60,
                scaling=scaling, downsampling=True, rand_state=4)
    result.append(score)
print(mean(result), '\n')

bci_type: learning , classifier: svm , scaling: True , coef_of_var: False
mutation_prob: 0.01 , num_of_generations: 100 , generation_size: 30 

[0 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0
 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0
 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0]

end test score mean:  0.9558823529411765 

[1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 0
 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1
 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0
 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0]

end test score mean:  0.8484848484848485 

[0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 1 1 0
 0 0 0 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 0

[0 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 1 0
 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1
 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0
 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 0]

end test score mean:  0.8695652173913043 

[1 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0
 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0
 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1
 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1]

end test score mean:  0.9545454545454546 

[1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0
 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 1
 1 1 0 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 0]

end test score mean:  0.9142857142857143 

[1 1 1 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 


end test score mean:  0.8865979381443299 

[1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 0
 0 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0
 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 0 0]

end test score mean:  0.7701149425287356 

[1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1
 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0
 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1]

end test score mean:  0.87 

[1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 1 0
 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1
 1 0 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0
 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1]

end test score mean:  0.7653061224489796 

[0 0 0 


end test score mean:  0.8969072164948454 

[0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 1
 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0]

end test score mean:  0.9012345679012346 

[0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1
 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1
 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0
 0 0 0 1 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 0]

end test score mean:  0.8703703703703703 

[1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1
 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1
 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0
 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0]

end test score mean:  0.917475728155

In [ ]:
# classifier: 'svm'/'knn'
# bci_type: 'learning'/'withdraw'
# ICA

# for bci_type in ['learning','withdraw']:
#     for classifier in ['poly2_svm']:
#         for scaling in [True]:
#             for coef_of_var in [False]:
#                 for mutation_prob in [0.01]:
#                     for num_of_generations in [100]:
#                         for generation_size in [30]:
#                             print('bci_type:', bci_type, ', classifier:', classifier, ', scaling:', scaling, ', coef_of_var:', coef_of_var)
#                             print('mutation_prob:', mutation_prob, ', num_of_generations:', num_of_generations, ', generation_size:', generation_size, '\n')
#                             result = []
#                             for i in range(10):
#                                 score = main(generation_size=generation_size, num_of_generations=num_of_generations, crossover_prob=1, mutation_prob=mutation_prob, 
#                                             classifier=classifier, bci_type=bci_type, coef_of_var = False, treshold = 60,
#                                             scaling=scaling, downsampling=True, rand_state=4)
#                                 result.append(score)
#                             print(mean(result), '\n')

In [ ]:
# nothing

# for bci_type in ['withdraw','learning']:
#     for classifier in ['knn','svm']:
#         for scaling in [True]:
#             for mutation_prob in [0.01]:
#                 for num_of_generations in [100]:
#                     for generation_size in [30]:
# #                         if not (bci_type=='withdraw' and classifier=='svm' and scaling==True and mutation_prob==0.01):
#                         print('bci_type:', bci_type, ', classifier:', classifier, ', scaling:', scaling)
#                         print('mutation_prob:', mutation_prob, ', num_of_generations:', num_of_generations, ', generation_size:', generation_size, '\n')
#                         result = []
#                         for i in range(10):
#                             score = main2(generation_size=generation_size, num_of_generations=num_of_generations, crossover_prob=1, mutation_prob=mutation_prob, 
#                                         classifier=classifier, bci_type=bci_type, coef_of_var = False, treshold = 60,
#                                         scaling=scaling, downsampling=True, rand_state=4)
#                             result.append(score)
#                         print(mean(result), '\n')

## ŚMIETNIK

In [ ]:
# zamiennik funkcji split_data jakbym robiła klasyfikacje wielokategorialną

# rest, test = load_data()
# n=3
# data = 

# # ta funkcja potrzebna przy 3 rodzajach eventów
# # def compensate(data, n):
# unique, counts = np.unique(data[:,-1], return_counts=True)
# print(counts)
# print(min(counts))

# # choose random rows
# idx1 = np.random.randint(data[data[:,-1] == unique[0]].shape[0], size=min(counts))
# idx2 = np.random.randint(data[data[:,-1] == unique[1]].shape[0], size=min(counts))
# if n == 3:
#     idx3 = np.random.randint(data[data[:,-1] == unique[2]].shape[0], size=min(counts))
# # print(idx1)
# # print(idx2)
# # print(idx3)

# # a = test[test[:,-1] == unique[0]]
# print(data[data[:,-1] == unique[0]][idx1,:].shape)
# print(data[data[:,-1] == unique[1]][idx2,:].shape)
# if n == 3:
#     print(data[data[:,-1] == unique[2]][idx3,:].shape)
# dict(zip(unique, counts))


In [ ]:
# stara funkcja split data

# def split_data(df, rand_state):

#     # split data to 4 groups: features and classes for errors and successful clicks
#     X_err = df[df[:,-1] == 0][:,:-1]
#     y_err = df[df[:,-1] == 0][:,-1]
#     X_succ = df[df[:,-1] == 1][:,:-1]
#     y_succ = df[df[:,-1] == 1][:,-1]

#     # 20% do ostatecznego testu, wyrównana liczba przypadków w grupach
#     X_err_rest, X_err_end_test, y_err_rest, y_err_end_test = train_test_split(X_err, y_err, test_size=0.2, random_state=rand_state)
#     X_succ_rest, X_succ_end_test, y_succ_rest, y_succ_end_test = train_test_split(X_succ, y_succ,
#                                                                                   test_size=y_err_end_test.shape[0],
#                                                                                   train_size=y_err_rest.shape[0],
#                                                                                   random_state=rand_state)

#     # w tym momencie dane do ostatecznego testu wychodzą posegregowane, najpierw błędy interfejsu, potem udane kliknięcia
#     # w k-NN to nie będzie miało znaczenia, a co w innych klasyfikatorach?
#     X_end_test = np.concatenate((X_err_end_test, X_succ_end_test))
#     y_end_test = np.concatenate((y_err_end_test, y_succ_end_test))

#     # sklejam pozostałe 80% które wykorzystam przy cross-walidacji
#     X_rest = np.concatenate((X_err_rest, X_succ_rest))
#     y_rest = np.concatenate((y_err_rest, y_succ_rest))

#     return X_rest, y_rest, X_end_test, y_end_test

In [ ]:
# # split data to 2 groups
# err_df = df[df['successful'] == 0]
# succ_df = df[df['successful'] == 1]

# # create features array and class array for errors
# X_err = np.array(err_df.drop(['successful'],1))
# y_err = np.array(err_df['successful'])

# # create features array and class array for successful clicks
# X_succ = np.array(succ_df.drop(['successful'],1))
# y_succ = np.array(succ_df['successful'])

In [ ]:
# funkcja tworząca pierwsze pokolenie
# jedynki oznaczają cechy sygnału przekazywane do klasyfikatora

# def first_gen(generation_size):
#     first_gen = []
#     for one in range(generation_size):
#         ind = []
#         for i in range(136):
#             ind.append(randint(0, 1))
#         first_gen.append(ind)
#     return first_gen

In [ ]:
# # funkcja oceniająca przystosowanie danego osobnika

# def eval(individual):
#     features = []
#     n_of_ones = 0
#     for i in range(136):
#         if individual[i] == 1:
#             features.append(i)
#     neigh = KNeighborsClassifier(n_neighbors=3)
#     neigh.fit(X_train[:, features], y_train)
#     accuracy = neigh.score(X_test[:, features], y_test)
#     return 136-len(features), accuracy, accuracy * (136-len(features)) - 50

In [ ]:
# # funkcja oceniająca całe pokolenie

# def evaluation(generation):
#     local_best_fitness = -100
#     local_best_accuracy = 0
#     local_n_of_zeros = 0
#     best_individual = []
#     gen_fitness = np.array([0])
#     for one in generation:
#         n_of_zeros, accuracy, fitness = eval(one)
#         fitness = eval(one)
#         if fitness > local_best_fitness:
#             local_best_fitness = fitness
#             local_best_accuracy = accuracy
#             local_n_of_zeros = n_of_zeros
#             best_individual = one
#         print(fitness)
#         np.append(gen_fitness, fitness)
# #     return gen_fitness, local_best_fitness, best_individual
#     return gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, best_individual

# f, lf, bi = evaluation(g)
# print(f)

In [ ]:
# # ruletka

# def roulette(generation, gen_fitness):
#     roulette_tab = []
#     prev = 0
#     for one in gen_fitness:
#         prev = one/sum(gen_fitness) + prev
#         roulette_tab.append(prev)
#     roulette_tab[-1] = 1
#     survivors = []
#     for i in range(generation_size):
#         random_num = random()
#         winner = 0
#         while random_num > roulette_tab[winner]:
#             winner += 1
#         survivors.append(generation[winner])
#     return survivors

In [ ]:
# def cross2ind(ind):
#     if np.random.rand() <= crossover_prob:
#         cut = np.random.randint(1,135)
#         print(cut)
#         print(ind)
#         idn = ind.reshape(2,136)
#         print('reshaped: ',ind)
#         temp = ind[1,:cut].copy()
#         ind[1,:cut] = ind[0,:cut]
#         ind[0,:cut] = temp
#         idn = ind.reshape(2,136)

In [ ]:
# # funkcja krzyżująca osobniki

# # def crossover(survivors):
# s = s.reshape(generation_size//2,272)
# # print(s)
# d = np.apply_along_axis(cross2ind, 1, s)

In [ ]:
# # funkcja mutująca wybrane geny wybranych osobników z pokolenia

# def mutation(descendants, mutation_prob):
#     for one in descendants:
#         if random() <= mutation_prob:
#             locus = randint(0,135)
# #             zamiana genu
#             one[locus] = 1 - one[locus]
#     return descendants